## Latent Dirichlet Allocation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
import re

from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

import warnings
warnings.filterwarnings("ignore")

In [2]:
import string
import spacy

In [3]:
#gensim for LDA
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  #dont skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
## Read the file 
df = pd.read_csv('turmeric_milk.csv')
df.head()

,name,reviewBody,reviewRating,datePublished,datePublishedRaw,probability,isVerified
0,Quick option for haldi doodh we moms always lo...,I always give my son haldi doodh at night befo...,5.0,2021-01-24T00:00:00,Reviewed in India on 24 January 2021,0.934556,NaN
1,Quick option for haldi doodh we moms always lo...,I always give my son haldi doodh at night befo...,5.0,2021-01-24T00:00:00,Reviewed in India on 24 January 2021,0.934556,NaN
2,Closing lid is of low quality.,I'm getting rating 2 as I'm unhappy with the w...,5.0,2021-03-13T00:00:00,Reviewed in India on 13 March 2021,0.966139,NaN
3,Closing lid is of low quality.,I'm getting rating 2 as I'm unhappy with the w...,2.0,2021-03-13T00:00:00,Reviewed in India on 13 March 2021,0.966139,NaN
4,Quick option for haldi doodh we moms always lo...,Your browser does not support HTML5 video. I a...,5.0,2021-01-24T00:00:00,Reviewed in India on 24 January 2021,0.851983,True


In [5]:
df.iloc[5,1]

'Your browser does not support HTML5 video. I always give my son haldi doodh at night before going to sleep. But considering the hectic lifestyle I have, it really becomes difficult to crush almonds, crush fresh haldi powder. So this one came as a promising option to opt for. The best part is that it has crushed almonds readymade and thus I do not need to make any separate preparations. Considering the portion size also, it is also promising as this pack is lasting more than a month.'

In [6]:
df = df.iloc[::-2][::-1] ##removing even index because they are all +ve
df_pos = df[df['reviewRating']>=4.0]
df_ne = df[df['reviewRating']<3.0]

In [7]:
def word_transform(df):
#     df = df.iloc[::2].reset_index(drop=True)
    content_df = df[['reviewBody','reviewRating']]
    #removing special charecters
    content_df['reviewBody'] = content_df['reviewBody'].apply(lambda x: re.sub(re.compile("[^\w\s]"), "", x))
    #removing space from front and back of the string
    content_df['reviewBody']= content_df['reviewBody'].apply(lambda x : x.strip())
    # Converting data into lower
    content_df['reviewBody']= content_df['reviewBody'].apply(lambda x : x.lower())
    return content_df 

content_df = word_transform(df_pos)

<>:5: DeprecationWarning: invalid escape sequence \w
<>:5: DeprecationWarning: invalid escape sequence \w
C:\Users\saika\AppData\Local\Temp/ipykernel_16220/4248882519.py:5: DeprecationWarning: invalid escape sequence \w
  content_df['reviewBody'] = content_df['reviewBody'].apply(lambda x: re.sub(re.compile("[^\w\s]"), "", x))


In [8]:
content_df.head()

,reviewBody,reviewRating
1,i always give my son haldi doodh at night befo...,5.0
5,your browser does not support html5 video i al...,5.0
7,loved this aryuvedic immunity booster haldi do...,5.0
9,almond flacks kaali mirch haldi etc are the ac...,5.0
11,this is s superb product from suffola it is a...,5.0


In [9]:
# tokenize using gensims simple_preprocess
def sent_to_words(sentences, deacc=True):  # deacc=True removes punctuations
    for sentence in sentences:
        yield(simple_preprocess(str(sentence)))

# conver to list
data=content_df['reviewBody'].values.tolist()
data_words=list(sent_to_words(data))

#print review
print(data_words[1])

['your', 'browser', 'does', 'not', 'support', 'html', 'video', 'always', 'give', 'my', 'son', 'haldi', 'doodh', 'at', 'night', 'before', 'going', 'to', 'sleep', 'but', 'considering', 'the', 'hectic', 'lifestyle', 'have', 'it', 'really', 'becomes', 'difficult', 'to', 'crush', 'almonds', 'crush', 'fresh', 'haldi', 'powder', 'so', 'this', 'one', 'came', 'as', 'promising', 'option', 'to', 'opt', 'for', 'the', 'best', 'part', 'is', 'that', 'it', 'has', 'crushed', 'almonds', 'readymade', 'and', 'thus', 'do', 'not', 'need', 'to', 'make', 'any', 'separate', 'preparations', 'considering', 'the', 'portion', 'size', 'also', 'it', 'is', 'also', 'promising', 'as', 'this', 'pack', 'is', 'lasting', 'more', 'than', 'month']


In [10]:
# create a list of stop words
# string.punctuation (from the 'string' module) contains a list of punctuations


from nltk.corpus import stopwords
stop_words= stopwords.words('english') + list(string.punctuation)

In [11]:
# functions for removing stopwords and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatization(texts,allowed_postags=['NOUN','ADJ','VERB','ADV']):
    """https://spacy.io/api/annotation"""
    texts_out=[]
    for sent in texts:
        doc=nlp(' '.join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
# call functions

# remove stop words
data_words_npstops= remove_stopwords(data_words)

# initialize spacy 'en' model use only tagger since we don;t need parsing or NER
# python -m spacy download en
# spacy.cli.download("en")
nlp=spacy.load('en_core_web_sm',disable=['parser', 'ner'])

# lemmatization keeping only noun, adj, vb, adv
data_lemmatized=lemmatization(data_words_npstops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[1])

['always', 'give', 'night', 'go', 'sleep', 'consider', 'hectic', 'lifestyle', 'really', 'become', 'difficult', 'crush', 'almond', 'crush', 'fresh', 'powder', 'one', 'come', 'promise', 'option', 'opt', 'good', 'part', 'crush', 'almond', 'readymade', 'thus', 'need', 'make', 'separate', 'preparation', 'consider', 'portion', 'size', 'also', 'also', 'promise', 'pack', 'last', 'month']


In [13]:
# compare the nostop, lemmatised version with the original one
# note that speakers is lemmatised to speaker; 


print(' '.join(data_words[3]), '\n')
print(' '.join(data_lemmatized[3]))

almond flacks kaali mirch haldi etc are the active ingredients and the base ingredient is milk powder it gives very good taste to the milkyou will get best flavour after leaving it for sometime in the warm milk 

almond flack active ingredient base ingredient milk powder give good taste get good flavour leave sometime warm milk


In [14]:
# create dictionary and corpus
# create dictionary
id2word=corpora.Dictionary(data_lemmatized)

#create corpus
corpus=[id2word.doc2bow(text) for text in data_lemmatized]

# sample


print(corpus[2])

[(0, 1), (9, 1), (15, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 2), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 3), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1)]


In [15]:
# human-readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('almond', 2),
  ('also', 2),
  ('always', 1),
  ('become', 1),
  ('come', 1),
  ('consider', 2),
  ('crush', 3),
  ('difficult', 1),
  ('fresh', 1),
  ('give', 1),
  ('go', 1),
  ('good', 1),
  ('hectic', 1),
  ('last', 1),
  ('lifestyle', 1),
  ('make', 1),
  ('month', 1),
  ('need', 1),
  ('night', 1),
  ('one', 1),
  ('opt', 1),
  ('option', 1),
  ('pack', 1),
  ('part', 1),
  ('portion', 1),
  ('powder', 1),
  ('preparation', 1),
  ('promise', 2),
  ('readymade', 1),
  ('really', 1),
  ('separate', 1),
  ('size', 1),
  ('sleep', 1),
  ('thus', 1)]]

In [16]:
# Build LDA model
lda_model= gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=10,random_state=100,\
                                          update_every=1,chunksize=100,passes=10,alpha='auto',per_word_topics=True)


# LDA Model building
lda_model= gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=10,random_state=100,\
                                          update_every=1,chunksize=100,passes=10,alpha='auto',per_word_topics=True)

In [17]:
# print topics
lda_model.print_topics()

[(0,
  '0.040*"product" + 0.040*"taste" + 0.040*"life" + 0.021*"time" + 0.021*"everyday" + 0.021*"tough" + 0.021*"great" + 0.021*"add" + 0.021*"amazing" + 0.021*"do"'),
 (1,
  '0.034*"taste" + 0.023*"follow" + 0.023*"milk" + 0.023*"product" + 0.023*"recommend" + 0.023*"turmeric" + 0.023*"time" + 0.023*"powder" + 0.012*"ingredient" + 0.012*"week"'),
 (2,
  '0.034*"ingredient" + 0.028*"good" + 0.028*"product" + 0.017*"get" + 0.017*"taste" + 0.017*"amazing" + 0.017*"herb" + 0.017*"goodness" + 0.017*"milk" + 0.017*"recommend"'),
 (3,
  '0.034*"drink" + 0.025*"health" + 0.024*"product" + 0.023*"bring" + 0.023*"use" + 0.023*"amazing" + 0.023*"aryuvedic" + 0.023*"power" + 0.013*"good" + 0.012*"saffron"'),
 (4,
  '0.041*"day" + 0.038*"immunity" + 0.037*"love" + 0.033*"black" + 0.028*"pepper" + 0.028*"part" + 0.028*"daily" + 0.028*"latte" + 0.028*"highly" + 0.028*"recommend"'),
 (5,
  '0.033*"really" + 0.033*"crush" + 0.029*"good" + 0.026*"also" + 0.022*"promise" + 0.022*"almond" + 0.022*"consi

In [18]:
# coherence score
coherence_model_lda=CoherenceModel(model=lda_model,texts=data_lemmatized,dictionary=id2word,coherence='c_v')
coherence_lda=coherence_model_lda.get_coherence()
print('\nCoherence Score:',coherence_lda)


Coherence Score: 0.45794528720569405


In [19]:
# visulaise the topics

pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim_models.prepare(lda_model,corpus,id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.173082  0.053793       1        1  34.788839
5      0.206406 -0.075472       2        1  20.482068
7     -0.010956  0.075141       3        1  10.169759
4     -0.111641 -0.200970       4        1   9.159436
9      0.008612  0.021253       5        1   7.105841
2      0.043363  0.026033       6        1   7.104227
1     -0.009623  0.008970       7        1   4.615925
3      0.022468  0.076128       8        1   4.514670
0      0.017946  0.021071       9        1   1.297277
6      0.006507 -0.005947      10        1   0.761958, topic_info=          Term       Freq      Total Category  logprob  loglift
78       taste  28.000000  28.000000  Default  30.0000  30.0000
61     product  23.000000  23.000000  Default  29.0000  29.0000
52    immunity  12.000000  12.000000  Default  28.0000  28.0000
49      health  10.000000  10.000000  Default  27.0000  27.0000
43       drink  13.000000  13.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
51         hot   0.019525   6.381576  Topic10  -6.1109  -0.9124
73     flavour   0.019553   9.955892  Topic10  -6.1094  -1.3557
148       free   0.019542   7.045424  Topic10  -6.1100  -1.0105
167      spice   0.019541   7.098671  Topic10  -6.1101  -1.0181
63   recommend   0.019496  11.161300  Topic10  -6.1123  -1.4729

[506 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
113       8  0.828935  absolutely
70        6  0.767569      active
177       1  0.580721         add
177       3  0.387147         add
368       7  0.827698      advice
...     ...       ...         ...
124       7  0.344938        well
175       1  0.888546    wellness
258       5  0.765561     worried
176       1  0.888534       worth
284       3  0.726862       yogue

[465 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 6, 8, 5, 10, 3, 2, 4, 1, 7])